<a href="https://colab.research.google.com/github/haizzzi/simple-nmt-simplied/blob/main/NTC_Exe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

# GPU가 있는지 확인
if torch.cuda.is_available():
    print("사용 가능한 GPU 수:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i} 이름:", torch.cuda.get_device_name(i))
else:
    print("GPU를 사용할 수 없습니다.")


사용 가능한 GPU 수: 1
GPU 0 이름: NVIDIA A100-SXM4-40GB


In [1]:
!pip install torch torchvision torchaudio
!pip install torch_optimizer
!pip install pytorch-ignite

In [ ]:
!wc -l /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv

147893 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv


이 코드는 train과 valid 나누는 코드인데 불필요함. train만 넣으면 학습 코드에서 자동으로 나눠줌

In [ ]:
!head -n 118314 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv > /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train1.tsv
!tail -n 29579 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv > /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.valid1.tsv


In [ ]:
!wc -l /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.*.tsv

   30000 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.test.tsv
  118314 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train1.tsv
  147893 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv
   29579 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.valid1.tsv
  325786 total


simple-ntc 학습하는 코드, rnn, cnn 같이 학습함. 이후 classify할 때 앙상블로 두개 모두 활용함. cnn 부분은 A100에서 gpu 사용이 안되고 cpu로 사용되는 현상이 발생해서 내부에 GPU관련 설정을 바꿔서 train2.py를 만들고 simple-ntc/trainer.py도 바꿔줌.
자세한 변경사항은 아래와 같이 정리했음
----------------------------------------------------------
원인: RNN 학습 후 같은 프로세스에서 CNN으로 전환 시, A100에서 cudnn.benchmark=True가 유지되어 Conv2d(1→128, kernel=(k, 256)) 형태에 대해 cuDNN이 첫 배치에서 과도한 커널 탐색을 수행(수분 정지). 초기 스텝에서 큰 그래디언트로 AMP 스케일러가 step을 스킵하면서 |g_param| 로그가 NaN으로 찍히는 현상도 동반.

조치: RNN→CNN 전환 직전에

gc.collect(), torch.cuda.empty_cache()로 메모리/캐시 정리,

torch.backends.cudnn.benchmark=False, torch.backends.cudnn.deterministic=True로 커널 탐색 차단 및 결정적 커널 고정,

학습 루프에 **AMP( torch.amp.autocast + GradScaler('cuda') )**와 gradient clipping(clip_grad_norm_) 적용,

|g_param| 로깅을 safe_grad_norm(비유한값 제외)으로 교체,

(옵션) TF32 허용, channels_last 메모리 포맷, CNN 하이퍼(윈도우/필터) 경량화.

(주의) 실제 로드되는 모듈 경로 고정(sys.path.insert(0, PROJECT_ROOT))로 최신 trainer.py가 임포트되도록 정리.

결과: CNN 첫 배치 정지 해소, NaN 로그 제거/안정화, A100에서 에폭 당 시간 대폭 단축(RNN 영향 없음).



In [3]:
!python /content/drive/MyDrive/nmt_fold/simple-ntc/train2.py --model_fn /content/drive/MyDrive/nmt_fold/data2/review.pth --train_fn /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv --gpu_id 0 --batch_size 128 --n_epochs 10 --word_vec_size 256 --dropout .3 --rnn --hidden_size 512 --n_layers 4 --cnn --window_sizes 3 4 5 6 7 8 --n_filters 128 128 128 128 128 128
#!python /content/drive/MyDrive/nmt_fold/simple-ntc/train2.py --model_fn /content/drive/MyDrive/nmt_fold/data2/review.pth --train_fn /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv --gpu_id 0 --batch_size 128 --n_epochs 1 --word_vec_size 256 --dropout .3  --rnn --hidden_size 512 --n_layers 4 --cnn --window_sizes 3 4 5 6 7 8 --n_filters 128 128 128 128 128 128


|train| = 118315 |valid| = 29578
|vocab| = 13004 |classes| = 2
RNNClassifier(
  (emb): Embedding(13004, 256)
  (rnn): LSTM(256, 512, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (generator): Linear(in_features=1024, out_features=2, bias=True)
  (activation): LogSoftmax(dim=-1)
)
Epoch 1 - |param|=1.84e+03 |g_param|=6.18e-01 loss=3.0927e-01 accuracy=0.8858
Validation - loss=2.7446e-01 accuracy=0.8941 best_loss=inf
Epoch 2 - |param|=1.84e+03 |g_param|=3.77e-01 loss=2.0730e-01 accuracy=0.9284
Validation - loss=2.0735e-01 accuracy=0.9242 best_loss=2.7446e-01
Epoch 3 - |param|=1.84e+03 |g_param|=3.39e-01 loss=1.9198e-01 accuracy=0.9345
Validation - loss=1.9923e-01 accuracy=0.9305 best_loss=2.0735e-01
Epoch 4 - |param|=1.84e+03 |g_param|=2.93e-01 loss=1.5261e-01 accuracy=0.9493
Validation - loss=1.9668e-01 accuracy=0.9319 best_loss=1.9923e-01
Epoch 5 - |param|=1.84e+03 |g_param|=2.95e-01 loss=1.3846e-01 accuracy=0.9554
Validation - loss=1.9493e-01 accuracy=0.9322 best_l